In [177]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import re 
from geopy.distance import geodesic
from datetime import datetime, timedelta, time, date
import matplotlib.pyplot as plt
# Part 2

In [178]:
# Define the inputs

# L: set of flights
flights_df = pd.read_excel('Assignment_Data/Group_4_P2.xlsx', sheet_name='Flight')#.set_index('Flight no.')
flights_list = flights_df['Flight Number'].to_list() 

# P: set of passenger itineraries
paths = pd.read_excel('Assignment_Data/Group_4_P2.xlsx', sheet_name='Itinerary').set_index('Itin No.')

# P_p: set of passenger itineraries with Recapture Rate from itinerary p
recapture_p = pd.read_excel('Assignment_Data/Group_4_P2.xlsx', sheet_name='Recapture Rate').set_index(['From Itinerary','To Itinerary'])
recapture_p.rename(columns={'From Itinerary': 'p', 'To Itinerary': 'r'}, inplace=True)

# K: set of aircraft types
aircraft_df = pd.read_excel('Assignment_Data/Group_4_P2.xlsx', sheet_name='Aircraft')
aircraft_df.rename(columns={'Type': 'AC Type'}, inplace=True)
aircraft_df.set_index('AC Type', inplace=True)
aircraft = aircraft_df.to_dict(orient='index')
ac_list = list(aircraft.keys())

# make a dictionary with the itinerary as the key and the rest as a sub-dictionary
paths = paths.to_dict(orient='index')
path_list = list(paths.keys())

# flights = flights_df.to_dict(orient='index')
recapture_p = recapture_p.to_dict(orient='index')

# For all paths if 'Leg 1' and 'Leg 2' are numbers then create a list with both legs else, drop the keys from the list, and create a new key called 'Legs'
# else just change the name of the key 'Leg 1' to 'Legs'
for key in path_list:
    legs = []
    if paths[key]['Leg 1'] != 0 and paths[key]['Leg 2'] != 0:
        legs.append(paths[key]['Leg 1'])
        legs.append(paths[key]['Leg 2'])
        paths[key]['Legs'] = legs
    elif paths[key]['Leg 1'] != 0:
        legs.append(paths[key]['Leg 1'])
        paths[key]['Legs'] = legs
    del paths[key]['Leg 1']
    del paths[key]['Leg 2']

# Define path 999 with a fare of 0 and a demand of 0 
paths[999] = {'Legs': [], 'Demand': 0, 'Fare': 0}

for k in aircraft:
    aircraft[k]['TAT'] = timedelta(minutes=aircraft[k]['TAT'])

flights = flights_df.merge(aircraft_df.reset_index()[['AC Type']], how='cross')

In [179]:
aircraft

{'A330': {'Units': 4, 'Seats': 210, 'TAT': datetime.timedelta(seconds=3000)},
 'A340': {'Units': 2, 'Seats': 220, 'TAT': datetime.timedelta(seconds=3600)},
 'B737': {'Units': 8, 'Seats': 120, 'TAT': datetime.timedelta(seconds=2100)},
 'B738': {'Units': 29, 'Seats': 140, 'TAT': datetime.timedelta(seconds=2100)}}

In [180]:
# Get the cost from the column named after the AC Type
flights['Cost'] = flights.apply(lambda row: row[row['AC Type']], axis=1)
flights.drop(columns=ac_list, inplace=True)
flights['Capacity'] = flights.apply(lambda row: aircraft[row['AC Type']]['Seats'], axis=1)

In [181]:
# Read the data distance file
distance_info = pd.read_csv('Assignment_Data/Group_4_Distances.csv')
distance_info.rename(columns={'Unnamed: 0': 'Origin'}, inplace=True)

# Create a dictionary with the distance info
distance = {}
for index, row in distance_info.iterrows():
    distance[row['Origin']] = row.to_dict()

# Remove the origin column from the distance
for i in distance:
    distance[i].pop('Origin', None)

In [182]:
# List of unique airports from Origin and Destination columns
airports = list(set(flights['ORG'].unique()).union(set(flights['DEST'].unique())))

In [183]:
misc_date = date(1,1,1)

# Drop rows with distance > range
flights['Arrival'] = flights.apply(lambda row: (datetime.combine(misc_date,row['Arrival']) + aircraft[row['AC Type']]['TAT']).time(), axis=1)
flights['Overnight'] = flights.apply(lambda row: row['Arrival'] < row['Departure'], axis=1)

In [184]:
# Make flights dictionary with main keys: AC Type, with a sub dictionary of flight numbers and each with flight details
flights_dict = {}
for i in flights_list:
    flights_dict[i] = flights[flights['Flight Number'] == i].set_index('AC Type').to_dict(orient='index')

flights = flights_dict

In [185]:
# Create an empty list to store the data
data = []
# Iterate over airports, aircraft types, and flights
for l in flights:
    for k in flights[l]:
            for n in airports:
                if flights[l][k]['ORG'] == n:
                    data.append([k, n, l, flights[l][k]['Departure'], 'Departure'])
                if flights[l][k]['DEST'] == n:
                    data.append([k, n, l, flights[l][k]['Arrival'], 'Arrival'])

# Create a dataframe with the data
events = pd.DataFrame(data, columns=['AC Type','Airport', 'Flight N', 'Time', 'D_A'])

# Add the TAT to the arrival times

events.sort_values(by=['AC Type', 'Airport', 'Time'], inplace=True)

# Reset the numbering of the events
events.reset_index(drop=True, inplace=True)

In [186]:
# For each airport and aircraft type i need to create a loop of ground arcs, each starting from the last event and ending at the next event, if there is no more events then the last ground arc is the overnight arc and it ends at the first event of the next day (first event of the next day is the first event of the same airport and aircraft type)

ground_arcs = pd.DataFrame(columns=['AC Type', 'Airport', 'Start Time', 'End Time'])

for k in ac_list:
    for n in airports:
        df = events[(events['AC Type'] == k) & (events['Airport'] == n)].sort_values(by=['Time'])
        for i in range(len(df)):
            if i == 0:
                ground_arcs = pd.concat([ground_arcs, pd.DataFrame({'AC Type': k, 'Airport': n, 'Start Time': [df.iloc[-1]['Time']], 'End Time': [df.iloc[i]['Time']]})], ignore_index=True)
            else:
                ground_arcs = pd.concat([ground_arcs, pd.DataFrame({'AC Type': k, 'Airport': n, 'Start Time': [df.iloc[i-1]['Time']], 'End Time': [df.iloc[i]['Time']]})], ignore_index=True)

ground_arcs.sort_values(by=['AC Type', 'Airport'], inplace=True)
# Drop rows if start time and end time are the same
ground_arcs = ground_arcs[~ground_arcs.apply(lambda row: (row['Start Time'] == row['End Time']), axis=1)]
ground_arcs['Arc ID'] = ground_arcs.groupby(['AC Type']).cumcount()
ground_arcs['Overnight'] = ground_arcs.apply(lambda row: (row['End Time'] < row['Start Time']), axis=1)
ground_arcs.sort_values(by='Arc ID', inplace=True)

In [187]:
# Create a nodes_df
nodes_df = ground_arcs[['AC Type', 'Airport', 'Start Time']].rename(columns={'Start Time': 'Time'})

# Add a count number for each row group by AC Type and Airport
nodes_df['Node ID'] = nodes_df.groupby(['AC Type']).cumcount()

# make a dictionary with the ac type as main key and the airport as secondary key with the node as tertiary key and the time as value
nodes = {}
for k in ac_list:
    nodes[k] = {}
    for n in airports:
        nodes[k][n] = {}
        for i in nodes_df[(nodes_df['AC Type'] == k) & (nodes_df['Airport'] == n)]['Node ID']:
            nodes[k][n][i] = {'Time': nodes_df[(nodes_df['AC Type'] == k) & 
                                               (nodes_df['Airport'] == n) & 
                                               (nodes_df['Node ID'] == i)]['Time'].values[0]}
            

In [188]:
# Add node id to the events
events.merge(nodes_df, how='left', on=['AC Type', 'Airport', 'Time']).sort_values(by=['AC Type', 'Airport'])

# Add a dictionary call departures and another one arrivals to the nodes dictionary at k,n,i+1 with 
# the events that have the same ac type, airport and time as the node
for k in ac_list:
    for n in airports:
        for i in nodes_df[(nodes_df['AC Type'] == k) & (nodes_df['Airport'] == n)]['Node ID']:
            nodes[k][n][i]['Departures'] = list(events[(events['AC Type'] == k) & (events['Airport'] == n) & (events['Time'] == nodes[k][n][i]['Time']) & (events['D_A'] == 'Departure')]['Flight N'])
            nodes[k][n][i]['Arrivals'] = list(events[(events['AC Type'] == k) & (events['Airport'] == n) & (events['Time'] == nodes[k][n][i]['Time']) & (events['D_A'] == 'Arrival')]['Flight N'])


In [189]:
# n+: ground arcs originating at any node n (start time)
# n-: ground arcs ending at any node n (end time)
n_plus = ground_arcs[['Airport', 'AC Type', 'Start Time', 'Arc ID']].rename(columns={'Start Time': 'Time'})
n_minus = ground_arcs[['Airport', 'AC Type', 'End Time', 'Arc ID']].rename(columns={'End Time': 'Time'})

# Add a dictionary call n+ and another one n- to the nodes dictionary at k,n,i+1 with 
# the events that have the same ac type, airport and time as the node
for k in ac_list:
    for n in airports:
        for i in nodes_df[(nodes_df['AC Type'] == k) & (nodes_df['Airport'] == n)]['Node ID']:
            nodes[k][n][i]['n+'] = list(n_plus[(n_plus['AC Type'] == k) & (n_plus['Airport'] == n) & (n_plus['Time'] == nodes[k][n][i]['Time'])]['Arc ID'])
            nodes[k][n][i]['n-'] = list(n_minus[(n_minus['AC Type'] == k) & (n_minus['Airport'] == n) & (n_minus['Time'] == nodes[k][n][i]['Time'])]['Arc ID'])

In [190]:
overnight_arcs = ground_arcs[ground_arcs['Overnight'] == True][['AC Type', 'Airport', 'Arc ID']]
overnight_flights = []
for l in flights:
    for k in flights[l]:
        if flights[l][k]['Overnight']:
            overnight_flights.append([k, l])

overnight_flights = pd.DataFrame(overnight_flights, columns=['AC Type', 'Flight no.'])

In [191]:
# s_ip: binary variable indicating whether flight i is in itinerary p
s_ip = {}
for i in flights_list:
    for p in paths:
        s_ip[i,999] = 0
        if i in paths[p]['Legs']:
            s_ip[i,p] = 1
        else:
            s_ip[i,p] = 0

# Q_i: unconstrained demand for flight i = sum s_ip * demand of itinerary p for p in P
Q_i = {}
for i in flights_list:
    Q_i[i] = 0
    for p in paths:
        Q_i[i] += s_ip[i,p] * paths[p]['Demand']

In [192]:
# Add entries to P_p for path 0 with a Recapture Rate of 1
for p in paths:
    recapture_p[p,999] = {'Recapture Rate': 1}
    recapture_p[999,p] = {'Recapture Rate': 0}

path_list = list(paths.keys())
flight_list = list(flights.keys())

In [193]:
plotiing = False
if plotiing:
    import random
    import plotly.graph_objects as go

    for k in ac_list:
        df = events[events['AC Type'] == k].sort_values(by=['Time'])
        fig = go.Figure()
        
        # Set marker color based on 'D_A' column
        marker_color = ['red' if d_a == 'Departure' else 'blue' for d_a in df['D_A']]
        
        # Add ground arcs
        ground_arcs_k = ground_arcs[ground_arcs['AC Type'] == k]
        for i, row in ground_arcs_k.iterrows():
            # Generate a random color
            random_color = '#' + ''.join(random.choices('0123456789ABCDEF', k=6))
            fig.add_shape(
                type="line",
                x0=row['Start Time'],
                y0=row['Airport'],
                x1=row['End Time'],
                y1=row['Airport'],
                line=dict(color=random_color, width=2)
            )
        
        fig.add_trace(go.Scatter(
            x=df['Time'], 
            y=df['Airport'], 
            mode='markers+text',
            marker=dict(color=marker_color),
            hovertemplate= '<b>Flight no.</b>: ' + df['Flight N'] + '<br>' ))
        
        
        fig.update_layout(
            title="Ground Arcs for AC Type " + k,
            xaxis_title="Time",
            yaxis_title="Airport",
        )
        fig.show()


In [194]:
initial = [999]

In [ ]:
# i need to make current_pairs with all the [x,999] pairs 
inital_pairs = []
for p in path_list:
    inital_pairs.append([p,999])

In [218]:
# Define the model
m = gp.Model('IFAM')

# Define the decision variables
# f[i,k] [RELAXED] 1 if flight arc i is assigned to aircraft type k, 0 otherwise
f = {}

# t_pr: number of passengers that would like to fly on itinerary p and are reallocated to itinerary r
t = {}

# y_ak = number of aircraft of type k on the ground arc a (relaxed integer with lowerbound 0)
y = {}

for p in path_list:
    for r in initial:
        t[p,r] = m.addVar(vtype=GRB.CONTINUOUS,lb=0,name='t_'+str(p)+'_'+str(r))

for i in flights_list:
    for k in ac_list:
        f[i, k] = m.addVar(vtype=GRB.CONTINUOUS, name='f_' + str(i) + '_' + str(k))

# y_ak = number of aircraft of type k on the ground arc a (integer with lowerbound 0)
y = {}
for k in ac_list:
    for a in list(ground_arcs[(ground_arcs['AC Type'] == k)]['Arc ID']):
        y[a, k] = m.addVar(vtype=GRB.CONTINUOUS,lb=0, name='y_' + str(a) + '_' + str(k))

m.update()
of = gp.quicksum(flights[i][k]['Cost'] * f[i,k]
                  for i in flights for k in ac_list)
of +=  gp.quicksum((paths[p]['Fare'] - recapture_p[(p,r)]['Recapture Rate'] * paths[r]['Fare']) * t[p,r] 
                  for r in initial for p in path_list)



# Define the objective function
m.setObjective(of, GRB.MINIMIZE)


# Define the constraints
# Constraint 1: Each flight is assigned to exactly one aircraft type
for i in flights:
    m.addConstr((gp.quicksum(f[i,k] for k in ac_list) == 1), name='one_ac')

# Constraint 2: The number of AC arriving = AC departing, for each type at each node
# y_n+_k + sum(f_i,k) = y_n-_k + sum(f_i,k)
for k in ac_list:
    for n in airports:
        for i in nodes[k][n]:
            m.addConstr((y[nodes[k][n][i]['n+'][0], k] + gp.quicksum(f[w,k] for w in nodes[k][n][i]['Departures']) == 
                         y[nodes[k][n][i]['n-'][0], k] + gp.quicksum(f[w,k] for w in nodes[k][n][i]['Arrivals']) ),
                         name='balance_' + str(i) + '_' + str(k) + '_' + str(n))

# Constraint 3: The number of overnight arcs + the number of overnight flights = the number of aircraft of each type 
# using overnight_arcs and overnight_flights
# sum(y_a,k) + sum(f_i,k) = number of aircraft of type k
for k in ac_list:
    m.addConstr((gp.quicksum(y[a, k] for a in list(overnight_arcs[(overnight_arcs['AC Type'] == k)]['Arc ID'])) + 
                 gp.quicksum(f[i,k] for i in list(overnight_flights[(overnight_flights['AC Type'] == k)]['Flight no.'])) == 
                 aircraft[k]['Units']), name='overnight_' + str(k))

# Constraint 4: # sum seats_k * f_ik -sum s_ip * t_pr - sum sum s_ip * brp * t_rp >= ds_i for all i but for r = 0 
m.addConstrs((gp.quicksum(aircraft[k]['Seats'] * f[i,k] for k in ac_list) +
              gp.quicksum(s_ip[i,p] * t[p,r] for p in path_list for r in initial) - 
              gp.quicksum(s_ip[i,p] * recapture_p[(r,p)]['Recapture Rate'] * t[p,r] for p in path_list for r in initial) >= 
              Q_i[i] for i in flight_list), name='π')

# Constraint 5: sum t_pr <= Dp for all p
m.addConstrs((gp.quicksum(t[p,r] for r in initial) <= paths[p]['Demand'] for p in path_list), name='σ')

# Update the model
m.update()

In [197]:
# Optimize the model but dont print the output
m.setParam('OutputFlag', 0)
m.optimize()
print('Objective value: %g' % (m.objVal/1000000))

Objective value: 5.49076


In [198]:
# Print the optimal objective value and the decision variables t_pr and the dual variables
print('Optimal objective value: %0.2f' % (m.objVal / 1000000))


print('Dual variables:')
for c in m.getConstrs():
    if c.Pi != 0 and (c.constrName[0] == 'π' or c.constrName[0] == 'σ'):
        print('%s = %g' % (c.ConstrName, c.Pi))

# Save dual variables in a dictionary
pi_dual = {}
for c in m.getConstrs():
    if c.constrName[0] == 'π':

        flight_num_pi = c.ConstrName[2:-1]
        pi_dual[flight_num_pi] = c.Pi

sigma_dual = {}
for c in m.getConstrs():
    if c.constrName[0] == 'σ':
        path_num_sigma = int(re.findall(r'\d+', c.ConstrName)[0])    
        sigma_dual[path_num_sigma] = c.Pi

# Patgs 999
for p in path_list:
    

Optimal objective value: 5.49
Dual variables:
π[AR1133] = 57
π[AR1240] = 241
π[AR1241] = 288.235
π[AR1248] = 173
π[AR1249] = 47
π[AR1252] = 100
π[AR1253] = 51
π[AR1256] = 108
π[AR1260] = 23
π[AR1261] = 47
π[AR1263] = 49
π[AR1293] = 54
π[AR1303] = 262
π[AR1408] = 25
π[AR1410] = 1
π[AR1446] = 201
π[AR1447] = 103
π[AR1451] = 93
π[AR1458] = 6
π[AR1461] = 70
π[AR1462] = 101
π[AR1470] = 107
π[AR1471] = 94
π[AR1472] = 308
π[AR1473] = 105
π[AR1474] = 98
π[AR1475] = 109
π[AR1476] = 94
π[AR1477] = 104
π[AR1478] = 281
π[AR1657] = 3.07906
π[AR1759] = 1
π[AR1833] = 48
π[AR1836] = 103
σ[9,999] = -142
σ[19,999] = -145
σ[37,999] = -136
σ[47,999] = -2
σ[48,999] = -121
σ[58,999] = -95
σ[65,999] = -103
σ[69,999] = -17
σ[71,999] = -11
σ[72,999] = -12
σ[80,999] = -110
σ[94,999] = -7
σ[98,999] = -4
σ[106,999] = -9
σ[111,999] = -3
σ[112,999] = -103
σ[114,999] = -101
σ[117,999] = -3
σ[118,999] = -3
σ[119,999] = -3
σ[123,999] = -203
σ[124,999] = -215
σ[125,999] = -302
σ[126,999] = -216
σ[127,999] = -216
σ[128,

In [199]:
tpr_prime = {}
# tpr = (fare_p - sum (π_i) for i being each flight in path p) - bpr * (fare_r - sum (π_j) for j being each flight in path p)) - σ_p
for p,r in recapture_p.keys():
    t_prime_pr = ((paths[p]['Fare'] - sum(pi_dual[i] for i in paths[p]['Legs'])) -
                    (recapture_p[(p,r)]['Recapture Rate']) *
                    (paths[r]['Fare'] - sum(pi_dual[j] for j in paths[r]['Legs'])) -
                    (sigma_dual[p]))
    if t_prime_pr < 0:
        tpr_prime[p,r] = t_prime_pr

new_pairs = list(tpr_prime.keys())

In [215]:
def PMF_n_iters(pi, sigma, n_iters, paths, path_list, flight_list, recapture_p, initial, current_pairs, iter=0):
    iter += 1
    print('Iteration number: ', iter)

    tpr_prime = {}
    # tpr = (fare_p - sum (π_i) for i being each flight in path p) - bpr * (fare_r - sum (π_j) for j being each flight in path p)) - σ_p
    for p,r in recapture_p.keys():
        t_prime_pr = ((paths[p]['Fare'] - sum(pi[i] for i in paths[p]['Legs'])) -
                        (recapture_p[(p,r)]['Recapture Rate']) *
                        (paths[r]['Fare'] - sum(pi[j] for j in paths[r]['Legs'])) -
                        (sigma[p]))
        if t_prime_pr < 0:
            tpr_prime[p,r] = t_prime_pr

    new_pairs = list(tpr_prime.keys())
    current_pairs.extend(new_pairs)

    if len(new_pairs) == 0:
        print('No new pairs, optimal solution found in previous iteration')
    
    if len(new_pairs) > 0:
        print('New pairs: ', new_pairs)
        print('Current pairs: ', current_pairs)
        m_n = gp.Model(str(iter)+'th PMF')
        # Define the decision variables
        # f[i,k] [RELAXED] 1 if flight arc i is assigned to aircraft type k, 0 otherwise
        f = {}

        # t_pr: number of passengers that would like to fly on itinerary p and are reallocated to itinerary r
        t = {}

        # y_ak = number of aircraft of type k on the ground arc a (relaxed integer with lowerbound 0)
        y = {}

        for p in path_list:
            for r in initial:
                t[p,r] = m_n.addVar(vtype=GRB.CONTINUOUS,lb=0,name='t_'+str(p)+'_'+str(r))

        for i in flights_list:
            for k in ac_list:
                f[i, k] = m_n.addVar(vtype=GRB.CONTINUOUS, name='f_' + str(i) + '_' + str(k))

        # y_ak = number of aircraft of type k on the ground arc a (integer with lowerbound 0)
        y = {}
        for k in ac_list:
            for a in list(ground_arcs[(ground_arcs['AC Type'] == k)]['Arc ID']):
                y[a, k] = m_n.addVar(vtype=GRB.CONTINUOUS,lb=0, name='y_' + str(a) + '_' + str(k))


        # Add the new columns to the RMP
        for p,r in current_pairs:
            t[p,r] = m_n.addVar(vtype=GRB.CONTINUOUS,lb=0,name='t_'+str(p)+'_'+str(r))
        
        m_n.update()

        # Update the objective function
        of =  gp.quicksum(flights[i][k]['Cost'] * f[i,k] for i in flights for k in ac_list)
        of += gp.quicksum((paths[p]['Fare'] - recapture_p[(p,r)]['Recapture Rate'] * paths[r]['Fare']) * t[p,r] for r in initial for p in path_list)
        of += gp.quicksum((paths[p]['Fare'] - recapture_p[(p,r)]['Recapture Rate'] * paths[r]['Fare']) * t[p,r] for p,r in current_pairs)
        m_n.setObjective(of, GRB.MINIMIZE)

        # Update the constraints
        # Constraint 1: Each flight is assigned to exactly one aircraft type
        for i in flights:
            m_n.addConstr((gp.quicksum(f[i,k] for k in ac_list) == 1), name='one_ac')
        
        # Constraint 2: The number of AC arriving = AC departing, for each type at each node
        # y_n+_k + sum(f_i,k) = y_n-_k + sum(f_i,k)
        for k in ac_list:
            for n in airports:
                for i in nodes[k][n]:
                    m_n.addConstr((
                        y[nodes[k][n][i]['n+'][0], k] + gp.quicksum(f[w,k] for w in nodes[k][n][i]['Departures']) == 
                        y[nodes[k][n][i]['n-'][0], k] + gp.quicksum(f[w,k] for w in nodes[k][n][i]['Arrivals']) ),
                        name='balance_' + str(i) + '_' + str(k) + '_' + str(n))
                    
        # Constraint 3: The number of overnight arcs + the number of overnight flights = the number of aircraft of each type 
        # using overnight_arcs and overnight_flights
        # sum(y_a,k) + sum(f_i,k) = number of aircraft of type k
        for k in ac_list:
            m_n.addConstr((
                gp.quicksum(y[a, k] for a in list(overnight_arcs[(overnight_arcs['AC Type'] == k)]['Arc ID'])) + 
                gp.quicksum(f[i,k] for i in list(overnight_flights[(overnight_flights['AC Type'] == k)]['Flight no.'])) == 
                aircraft[k]['Units']), name='overnight_' + str(k))
    
        # Constraint 4: Each flight is assigned to exactly one aircraft type
        m_n.addConstrs((
            gp.quicksum(aircraft[k]['Seats'] * f[i,k] for k in ac_list) +
            gp.quicksum(s_ip[i, p] * t[p,r] for p in path_list for r in initial) +
            gp.quicksum(s_ip[i, p] * t[p,r] for p, r in current_pairs) -
            gp.quicksum(s_ip[i, r] * recapture_p[(p, r)]['Recapture Rate'] * t[p,r] for p,r in current_pairs) >=
            Q_i[i] for i in flight_list), name='π')

        # Constraint 5.1: sum t_pr <= Dp for all p
        
        m_n.addConstrs((gp.quicksum(t[p,r] for r in initial) <= paths[p]['Demand'] for p in path_list), name='σ')

        # Update the model
        m_n.update()

        # Solve the model but dont show the output
        m_n.Params.OutputFlag = 0
        m_n.optimize()

        # Print the optimal objective value and the decision variables t_pr and the dual variables
        print('Optimal objective value: %g' % m_n.objVal)

        print('\nDual variables:')
        for c in m_n.getConstrs():
            if c.Pi != 0 and (c.constrName[0] == 'π' or c.constrName[0] == 'σ'):
                print('%s = %g' % (c.ConstrName, c.Pi))

        # Save dual variables in a dictionary
        pi_new = {}
        for c in m_n.getConstrs():
            if c.constrName[0] == 'π':
                flight_num_pi = c.ConstrName[2:-1]
                pi_new[flight_num_pi] = c.Pi

        sigma_new = {}
        for c in m_n.getConstrs():
            if c.constrName[0] == 'σ':
                path_num_sigma = int(re.findall(r'\d+', c.ConstrName)[0])    
                sigma_new[path_num_sigma] = c.Pi
        
        print('End of iteration number: ', iter, '\n')
        if iter == n_iters:
            print('Max number of iterations reached')
            return ":("
        
        PMF_n_iters(pi = pi_new,
                    sigma = sigma_new, 
                    n_iters = n_iters, 
                    paths = paths, 
                    path_list = path_list, 
                    flight_list = flight_list, 
                    recapture_p = recapture_p, 
                    initial = initial, 
                    current_pairs = current_pairs, 
                    iter=iter)

In [216]:
PMF_n_iters(pi_dual, sigma_dual, n_iters=20, paths=paths, path_list=path_list, flight_list=flight_list, recapture_p = recapture_p, initial = initial, current_pairs = [], iter=0)

Iteration number:  1
New pairs:  [(115, 120), (117, 120), (122, 68), (133, 163), (134, 164), (156, 158), (176, 173), (338, 279), (367, 285), (376, 291)]
Current pairs:  [(115, 120), (117, 120), (122, 68), (133, 163), (134, 164), (156, 158), (176, 173), (338, 279), (367, 285), (376, 291)]
Optimal objective value: 5.49004e+06

Dual variables:
π[AR1133] = 57
π[AR1240] = 241
π[AR1241] = 282.635
π[AR1248] = 173
π[AR1249] = 47
π[AR1252] = 100
π[AR1253] = 51
π[AR1256] = 108
π[AR1260] = 23
π[AR1261] = 47
π[AR1263] = 49
π[AR1293] = 54
π[AR1303] = 262
π[AR1408] = 19.55
π[AR1410] = 1
π[AR1446] = 201
π[AR1447] = 103
π[AR1451] = 93
π[AR1455] = 1
π[AR1458] = 6
π[AR1461] = 53.83
π[AR1462] = 101
π[AR1470] = 107
π[AR1471] = 94
π[AR1472] = 303.04
π[AR1473] = 104.361
π[AR1474] = 98
π[AR1475] = 108
π[AR1476] = 94
π[AR1477] = 104
π[AR1478] = 281
π[AR1657] = 3.07906
π[AR1759] = 1
π[AR1833] = 45
π[AR1836] = 101
σ[9,999] = -142
σ[19,999] = -145
σ[37,999] = -136
σ[47,999] = -2
σ[48,999] = -121
σ[58,999] = -95


In [61]:
for k in ac_list:
    for a in list(overnight_arcs[(overnight_arcs['AC Type'] == k)]['Arc ID']):
        print(a, k, y[a, k].varName, y[a, k].x)
    for i in list(overnight_flights[(overnight_flights['AC Type'] == k)]['Flight no.']):
        print(i, k, f[i,k].varName, f[i,k].x) 
    print(aircraft[k]['Units'])
    print('\n')
    

0 A330 y_0_A330 0.0
12 A330 y_12_A330 0.0
20 A330 y_20_A330 0.0
22 A330 y_22_A330 0.0
24 A330 y_24_A330 0.0
46 A330 y_46_A330 0.0
50 A330 y_50_A330 0.0
54 A330 y_54_A330 0.0
58 A330 y_58_A330 0.0
62 A330 y_62_A330 0.0
70 A330 y_70_A330 0.0
80 A330 y_80_A330 0.0
101 A330 y_101_A330 0.0
103 A330 y_103_A330 0.0
107 A330 y_107_A330 0.0
109 A330 y_109_A330 0.0
125 A330 y_125_A330 0.0
128 A330 y_128_A330 0.0
140 A330 y_140_A330 0.0
146 A330 y_146_A330 0.0
150 A330 y_150_A330 0.0
156 A330 y_156_A330 0.0
319 A330 y_319_A330 0.0
325 A330 y_325_A330 0.0
335 A330 y_335_A330 0.0
351 A330 y_351_A330 0.0
353 A330 y_353_A330 0.0
355 A330 y_355_A330 0.0
370 A330 y_370_A330 0.0
372 A330 y_372_A330 0.0
374 A330 y_374_A330 0.0
376 A330 y_376_A330 0.0
378 A330 y_378_A330 0.0
AR1132 A330 f_AR1132_A330 1.0
AR1133 A330 f_AR1133_A330 1.0
AR1160 A330 f_AR1160_A330 0.0
AR1161 A330 f_AR1161_A330 0.0
AR1254 A330 f_AR1254_A330 0.0
AR1257 A330 f_AR1257_A330 0.0
AR1302 A330 f_AR1302_A330 0.0
AR1303 A330 f_AR1303_A33

In [100]:
for w in ac_list:
    print('Flights for AC Type ' + w)
    for i in flights_list:
        for k in ac_list:
            if f[i,k].x > 0 and k == w:
                print(i," | ", f[i,k].x," | ", ('Cost = %0.2f' % (flights[i][k]['Cost'] * f[i,k].x / 1000)))
    print('\n')

# Count the non null values of f
total_flights = 0
for v in m.getVars():
    if v.x != 0 and v.varName[0] == 'f':
        total_flights += v.x
print('Total flights: ' + str(total_flights))
print(sum(f[i,k].x for i in flights for k in ac_list))


Flights for AC Type A330
AR1132  |  1.0  |  Cost = 19.80
AR1133  |  1.0  |  Cost = 20.19
AR1322  |  1.0  |  Cost = 18.40
AR1323  |  1.0  |  Cost = 18.13
AR1376  |  1.0  |  Cost = 17.35
AR1377  |  1.0  |  Cost = 17.40
AR1920  |  1.0  |  Cost = 19.54
AR1921  |  1.0  |  Cost = 19.54


Flights for AC Type A340
AR1240  |  0.28750000000000003  |  Cost = 4.51
AR1241  |  0.2875  |  Cost = 4.59
AR1302  |  0.5  |  Cost = 11.82
AR1303  |  0.5  |  Cost = 11.41
AR1304  |  1.0  |  Cost = 21.73
AR1305  |  1.0  |  Cost = 22.13
AR1471  |  0.21249999999999997  |  Cost = 2.94
AR1472  |  0.21249999999999997  |  Cost = 2.86
AR1473  |  0.21249999999999997  |  Cost = 2.99
AR1474  |  0.21249999999999997  |  Cost = 2.90
AR1475  |  0.5  |  Cost = 7.02
AR1476  |  0.5  |  Cost = 6.83


Flights for AC Type B737
AR1140  |  1.0  |  Cost = 8.75
AR1402  |  1.0  |  Cost = 9.92
AR1403  |  1.0  |  Cost = 9.47
AR1406  |  1.0  |  Cost = 9.92
AR1412  |  1.0000000000000004  |  Cost = 9.92
AR1413  |  1.0000000000000004  |  Co

In [ ]:
# Make a df with AC grounded at overnight on each airport
grounded_overnight = pd.DataFrame(columns=ac_list, index=airports)
overnight_arcs['count'] = overnight_arcs.apply(lambda row: y[row['Arc ID'], row['AC Type']].x, axis=1)

# Reorganize the df, make the columns the unique AC types and the rows the unique airports and the values the count
for k in ac_list:
    grounded_overnight[k] = overnight_arcs[overnight_arcs['AC Type'] == k].groupby(['Airport']).sum()['count']

# Fill the NaN values with 0
grounded_overnight.fillna(0, inplace=True)
grounded_overnight

,A310,A320
BOS,1.0,0.0
OPO,0.0,1.0
YTO,0.0,0.0
PDL,1.0,1.0
FNC,0.0,0.0
LIS,1.0,1.0


In [ ]:
for k in ac_list:
    for n in airports:
        for i in nodes[k][n]:
            # val = 0
            s = str(i) + " | " + str(k) + " | "  + str(n)+ " | " +str(nodes[k][n][i]['Time']) + " | " 
            s += str(y[nodes[k][n][i]['n+'][0], k].varName) + ' (n+)'
            # val += y[nodes[k][n][i]['n+'][0], k].varName
            for w in nodes[k][n][i]['Departures']:
                s += ' + ' + str(f[w, k].varName) + ' (f+)'
                # val += f[w, k].varName
            s += ' = ' + str(y[nodes[k][n][i]['n-'][0], k].varName)  + ' (n-)'
            # val -= y[nodes[k][n][i]['n-'][0], k].varName
            for w in nodes[k][n][i]['Arrivals']:
                s += ' + ' + str(f[w, k].varName) + ' (f-)'
                # val -= f[w, k].varName
            # s += ' = ' + str(val)
            print(s)

0 | A310 | BOS | 22:25:00 | y_0_A310 (n+) = y_1_A310 (n-) + f_S4221_A310 (f-)
1 | A310 | BOS | 03:15:00 | y_1_A310 (n+) + f_S422X_A310 (f+) = y_0_A310 (n-)
11 | A310 | OPO | 21:50:00 | y_11_A310 (n+) = y_14_A310 (n-) + f_S4212_A310 (f-)
12 | A310 | OPO | 12:25:00 | y_12_A310 (n+) = y_11_A310 (n-) + f_S4320_A310 (f-)
13 | A310 | OPO | 13:30:00 | y_13_A310 (n+) + f_S4321_A310 (f+) = y_12_A310 (n-)
14 | A310 | OPO | 19:05:00 | y_14_A310 (n+) + f_S4213_A310 (f+) = y_13_A310 (n-)
31 | A310 | YTO | 02:45:00 | y_31_A310 (n+) + f_S4223_A310 (f+) = y_32_A310 (n-)
32 | A310 | YTO | 00:25:00 | y_32_A310 (n+) = y_31_A310 (n-) + f_S4222_A310 (f-)
15 | A310 | PDL | 22:05:00 | y_15_A310 (n+) + f_S4128_A310 (f+) = y_30_A310 (n-) + f_S4213_A310 (f-)
16 | A310 | PDL | 08:40:00 | y_16_A310 (n+) = y_15_A310 (n-) + f_S422X_A310 (f-)
17 | A310 | PDL | 09:00:00 | y_17_A310 (n+) = y_16_A310 (n-) + f_S4223_A310 (f-)
18 | A310 | PDL | 09:25:00 | y_18_A310 (n+) + f_S4120_A310 (f+) = y_17_A310 (n-) + f_S4121_A310

In [ ]:
for k in ac_list:
    for n in airports:
        for i in nodes[k][n]:
            val = 0
            s = str(i) + " | " + str(k) + " | "  + str(n)+ " | " +str(nodes[k][n][i]['Time']) + " | " 
            s += str(y[nodes[k][n][i]['n+'][0], k].x) + ' (n+)'
            val += y[nodes[k][n][i]['n+'][0], k].x
            for w in nodes[k][n][i]['Departures']:
                s += ' + ' + str(f[w, k].x) + ' (f+)'
                val += f[w, k].x
            s += ' = ' + str(y[nodes[k][n][i]['n-'][0], k].x)  + ' (n-)'
            val -= y[nodes[k][n][i]['n-'][0], k].x
            for w in nodes[k][n][i]['Arrivals']:
                s += ' + ' + str(f[w, k].x) + ' (f-)'
                val -= f[w, k].x
            s += ' | Total ' + str(val)
            print(s)

0 | A310 | BOS | 22:25:00 | 1.0 (n+) = 0.0 (n-) + 1.0 (f-) | Total 0.0
1 | A310 | BOS | 03:15:00 | 0.0 (n+) + 1.0 (f+) = 1.0 (n-) | Total 0.0
11 | A310 | OPO | 21:50:00 | 0.0 (n+) = 0.0 (n-) + 0.0 (f-) | Total 0.0
12 | A310 | OPO | 12:25:00 | 0.0 (n+) = 0.0 (n-) + -0.0 (f-) | Total 0.0
13 | A310 | OPO | 13:30:00 | 0.0 (n+) + -0.0 (f+) = 0.0 (n-) | Total 0.0
14 | A310 | OPO | 19:05:00 | 0.0 (n+) + 0.0 (f+) = 0.0 (n-) | Total 0.0
31 | A310 | YTO | 02:45:00 | -0.0 (n+) + 1.0 (f+) = 1.0 (n-) | Total 0.0
32 | A310 | YTO | 00:25:00 | 1.0 (n+) = -0.0 (n-) + 1.0 (f-) | Total 0.0
15 | A310 | PDL | 22:05:00 | 1.0 (n+) + -0.0 (f+) = 1.0 (n-) + 0.0 (f-) | Total 0.0
16 | A310 | PDL | 08:40:00 | 2.0 (n+) = 1.0 (n-) + 1.0 (f-) | Total 0.0
17 | A310 | PDL | 09:00:00 | 3.0 (n+) = 2.0 (n-) + 1.0 (f-) | Total 0.0
18 | A310 | PDL | 09:25:00 | 2.0 (n+) + 1.0 (f+) = 3.0 (n-) + -0.0 (f-) | Total 0.0
19 | A310 | PDL | 09:35:00 | 2.0 (n+) + -0.0 (f+) = 2.0 (n-) | Total 0.0
20 | A310 | PDL | 10:35:00 | 1.0 (n+)